# The 20 newsgroups topic analysis

Instead of repeating the IMDB sentiment analysis from the lesson (because frankly, I'm a little bored with sentiment analysis), I will attempt to apply a similar approach to deep-learning NLP classification to a dataset a coworker has recently been messing around with in `scikit-learn`: `sklearn.datasets.fetch_20newsgroups`.

http://people.csail.mit.edu/jrennie/20Newsgroups/

## Setup data

In [1]:
import os
current_dir = os.getcwd()

LESSON_HOME_DIR = current_dir + '/'
DATA_HOME_DIR = LESSON_HOME_DIR + 'data/'

DATASET_DIR = DATA_HOME_DIR + '20_newsgroup/'
MODEL_DIR = DATASET_DIR + 'models/'

In [2]:
if not os.path.exists(MODEL_DIR):
    os.mkdir(DATASET_DIR)
    os.mkdir(MODEL_DIR)

In [3]:
from sklearn.datasets import fetch_20newsgroups

category_subset = [
    'alt.atheism',
    'comp.graphics',
    'comp.os.ms-windows.misc',
    'soc.religion.christian',
]

newsgroups = fetch_20newsgroups(
    subset = 'all',
    categories = category_subset,
    shuffle = True,
    remove = ('headers', 'footers', 'quotes'))

In [4]:
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'soc.religion.christian']

`target_names` are as requested

In [5]:
newsgroups.filenames.shape, newsgroups.target.shape, len(newsgroups.data)

((3754,), (3754,), 3754)

Keras implements `get_word_index()` for the IMDB dataset, which returns an dictionary of word->index derived from a json file hosted on Amazon S3.

It seems bizarre to me to host this when you can easily create it on-demand... anyway, sklearn doesn't provide this. So let's create our own index with `keras.preprocessing.text.Tokenizer` (https://keras.io/preprocessing/text/).

In [6]:
import keras.preprocessing.text
import string

# Workaround to add "Unicode support for keras.preprocessing.text"
# (https://github.com/fchollet/keras/issues/1072#issuecomment-295470970)
def text_to_word_sequence(text,
                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                          lower=True, split=" "):
    if lower: text = text.lower()
    if type(text) == unicode:
        translate_table = {ord(c): ord(t) for c,t in zip(filters, split*len(filters)) }
    else:
        translate_table = string.maketrans(filters, split * len(filters))
    text = text.translate(translate_table)
    seq = text.split(split)
    return [i for i in seq if i]
    
keras.preprocessing.text.text_to_word_sequence = text_to_word_sequence

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [7]:
from keras.preprocessing.text import Tokenizer

vocab_size = 20000

tokenizer = Tokenizer(nb_words=vocab_size)
tokenizer.fit_on_texts(newsgroups.data) # builds the word index
sequences = tokenizer.texts_to_sequences(newsgroups.data)

In [8]:
word_index = tokenizer.word_index

In [9]:
len(word_index)

72905

Reverse the `word_index` with `idx2word`.

In [10]:
idx2word = {v: k for k, v in word_index.iteritems()}

Let's take a look at the first review, both as a list of indices and as text reconstructed from the indices.

In [11]:
', '.join(map(str, sequences[0]))

'24, 2, 60, 566, 52, 20, 4829, 3, 389, 2, 3000, 1339, 2, 155, 386, 84, 901, 76, 4, 115, 24, 2, 88, 566, 76, 92, 402, 3, 525, 12351, 101, 2, 155, 385, 6, 1685, 2454, 236, 93, 182, 118, 1350, 335, 7, 108, 5725, 2764, 6, 3000, 8981, 20, 396, 3, 118, 127, 2454, 6, 158, 182, 90, 4499, 14, 129, 11, 2274, 81, 4808, 234, 219, 92, 23, 10264, 5605, 6, 720, 10, 3000, 61, 783, 5464, 8, 5725, 7, 1736, 3, 239, 3, 1464, 55, 2, 579, 5464, 214, 701, 45, 91, 23, 2, 1802, 4, 11819, 5, 1464, 10, 2, 2694, 74, 318, 10232, 129, 386, 436, 5, 10837, 24, 11, 30, 3, 102, 1641, 11, 114, 332, 8, 2, 1598, 5, 858, 537, 9, 11, 1237, 96, 386, 95, 17, 23, 3, 952, 26, 8, 9, 15584, 121, 23, 219, 9951, 9845, 61, 2104, 95, 96, 5725, 11, 8, 5, 2828, 3, 2501, 197, 4, 127, 863, 720, 200, 102, 337, 127, 1576, 1021, 93, 15, 495, 2, 2157, 4, 94, 396, 3, 5777, 127, 9605, 720, 182, 118, 614, 2221, 60, 261, 3, 583, 8, 9, 135, 2, 154, 1464, 8, 210, 1464'

In [12]:
idx2word[24]

u'on'

In [13]:
' '.join([idx2word[o] for o in sequences[0]])

u"on the one hand there are advantages to having the liturgy stay the same john has described some of these on the other hand some people seem to start tuning out the same old and pay attention better when things get changed around i think innovative priests and liturgy committees are trying to get our attention and make things more meaningful for us it drives me crazy too different people have differing preferences and needs in liturgy my local parish is innovative i prefer to go to mass at the next parish over sometimes we don't have the option of attending a mass in the style which best suits us john put a smiley on it but to just offer it up probably is the solution a related issue that it sounds like john does not have to deal with is that spouses may have different liturgical tastes my husband does like innovative it is a challenge to meet both of our spiritual needs without just going our separate ways when you include the factor of also trying to satisfy our children's needs th

In [14]:
newsgroups.target[0], newsgroups.target_names[newsgroups.target[0]]

(3, 'soc.religion.christian')

Distribution of the lengths of sentences:

In [15]:
import numpy as np

lens = np.array(map(len, newsgroups.data))
(lens.max(), lens.min(), lens.mean())

(158791, 0, 1493.157432072456)

Weird that there are sentences with 0 sequences (words) in them...

In [16]:
# get indices of arrays that do NOT satisfy np.nonzero
nonzero_indices = np.unique(np.nonzero(sequences)[0])
zero_indices = set(range(len(sequences))).difference(nonzero_indices)
len(zero_indices)

101

So there are 101 sentences with no words. E.g.

In [17]:
sequences[64], newsgroups.target_names[newsgroups.target[64]]

([], 'alt.atheism')

...sure.

Pad (with zero) or truncate each sentence to make consistent length.

In [18]:
from keras.preprocessing import sequence

seq_len = 1000

data = sequence.pad_sequences(sequences, maxlen=seq_len, value=0)

In [19]:
data[:10]

array([[    0,     0,     0, ...,     8,   210,  1464],
       [    0,     0,     0, ...,  3162,     8,    11],
       [    0,     0,     0, ...,     2,   318,  1142],
       ..., 
       [    0,     0,     0, ...,    47,     7,   740],
       [ 1070,    11,     8, ...,  2565,   356,   129],
       [    0,     0,     0, ..., 16529,   364,  8254]], dtype=int32)

Finally, let's turn the labels into categorical information.

In [20]:
from keras.utils.np_utils import to_categorical

newsgroups.target = to_categorical(np.asarray(newsgroups.target))

In [21]:
data.shape, newsgroups.target.shape

((3754, 1000), (3754, 4))

Split data into train-test.

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, newsgroups.target, test_size=0.33)

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2515, 1000), (1239, 1000), (2515, 4), (1239, 4))

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [24]:
vocab_size, seq_len

(20000, 1000)

In [25]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import Adam

# input_length => 1500-word reviews, 32 floats per word
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(len(newsgroups.target_names), activation='softmax')])

In [26]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 1000, 32)      640000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 32000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           3200100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [27]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 0s - loss: 1.4231 - acc: 0.2692 - val_loss: 1.3441 - val_acc: 0.3430
Epoch 2/10
2515/2515 [==============================] - 0s - loss: 1.3207 - acc: 0.3340 - val_loss: 1.2871 - val_acc: 0.4092
Epoch 3/10
2515/2515 [==============================] - 0s - loss: 1.2117 - acc: 0.4203 - val_loss: 1.0628 - val_acc: 0.5214
Epoch 4/10
2515/2515 [==============================] - 0s - loss: 0.8870 - acc: 0.5730 - val_loss: 0.7685 - val_acc: 0.6207
Epoch 5/10
2515/2515 [==============================] - 0s - loss: 0.5728 - acc: 0.7670 - val_loss: 0.7013 - val_acc: 0.6618
Epoch 6/10
2515/2515 [==============================] - 0s - loss: 0.3622 - acc: 0.8823 - val_loss: 0.6265 - val_acc: 0.7094
Epoch 7/10
2515/2515 [==============================] - 0s - loss: 0.2230 - acc: 0.9427 - val_loss: 0.6085 - val_acc: 0.7320
Epoch 8/10
2515/2515 [==============================] - 0s - loss: 0.1432 - a

Is a mid-70s validation accuracy.. Good? Bad?

Here are some accuracies [from an official `sklearn` example](http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html) that classifies documents by topics using a bag-of-words approach:

```
[('RidgeClassifier', 0.89726533628972649),
 ('Perceptron', 0.88543976348854403),
 ('PassiveAggressiveClassifier', 0.90613451589061345),
 ('KNeighborsClassifier', 0.85809312638580926),
 ('RandomForestClassifier', 0.83813747228381374),
 ('LinearSVC', 0.90022172949002222),
 ('SGDClassifier', 0.90096082779009612),
 ('LinearSVC', 0.87287509238728755),
 ('SGDClassifier', 0.88543976348854403),
 ('SGDClassifier', 0.89874353288987441),
 ('NearestCentroid', 0.85513673318551364),
 ('MultinomialNB', 0.90022172949002222),
 ('BernoulliNB', 0.88396156688839611),
 ('Pipeline', 0.8810051736881005)]
 
 mean: 0.88311688311688319
 ```

So, not a good result in comparison with much simpler approaches. Training accuracy is high, but testing accuracy is much poorer.

As a sanity check, I also ran code from [`pretrained_word_embeddings.py`](https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py) (from Keras's examples repository) which also runs against `20_newsgroups` (not the `sklearn` version though), and it was able to achieve:

    loss: 0.3784 - acc: 0.8734 - val_loss: 0.9177 - val_acc: 0.7257
after 10 epochs - again, not as accurate as the 'shallow', bag-of-words models - but comparable to the results I'm receiving here.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [28]:
from keras.layers.convolutional import Convolution1D, MaxPooling1D

conv1 = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2),
    Dropout(0.4),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.4),
    MaxPooling1D(5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.7),
    Dense(len(newsgroups.target_names), activation='softmax')])

In [29]:
from keras.optimizers import RMSprop

conv1.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 1000, 100)     2000000     embedding_input_2[0][0]          
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 1000, 100)     0           embedding_2[0][0]                
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 996, 128)      64128       dropout_2[0][0]                  
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 996, 128)      0           convolution1d_1[0][0]            
___________________________________________________________________________________________

In [30]:
conv1.optimizer.lr.get_value().item()

0.0010000000474974513

In [31]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 1.3843 - acc: 0.3097 - val_loss: 1.3530 - val_acc: 0.3672
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 1.3072 - acc: 0.3634 - val_loss: 1.3008 - val_acc: 0.3995
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 1.1637 - acc: 0.4163 - val_loss: 1.0546 - val_acc: 0.5044
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.8981 - acc: 0.5507 - val_loss: 0.8433 - val_acc: 0.6086
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.6943 - acc: 0.6791 - val_loss: 0.7239 - val_acc: 0.6844
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.5465 - acc: 0.7678 - val_loss: 0.6518 - val_acc: 0.7264
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.3971 - acc: 0.8457 - val_loss: 0.5879 - val_acc: 0.7417
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.3175 - a

In [32]:
conv1.optimizer.lr=0.01

In [33]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=4, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/4
2515/2515 [==============================] - 4s - loss: 0.2226 - acc: 0.9181 - val_loss: 0.5544 - val_acc: 0.7676
Epoch 2/4
2515/2515 [==============================] - 4s - loss: 0.1712 - acc: 0.9308 - val_loss: 0.6017 - val_acc: 0.7797
Epoch 3/4
2515/2515 [==============================] - 4s - loss: 0.1701 - acc: 0.9352 - val_loss: 0.6385 - val_acc: 0.7643
Epoch 4/4
2515/2515 [==============================] - 4s - loss: 0.1332 - acc: 0.9503 - val_loss: 0.6191 - val_acc: 0.7772


In [34]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=1, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/1
2515/2515 [==============================] - 4s - loss: 0.1232 - acc: 0.9507 - val_loss: 0.6244 - val_acc: 0.7772


A good improvement over the previous model.

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [35]:
from keras.utils.data_utils import get_file

def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [36]:
from utils import load_array
import pickle

def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [37]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.100d'))

Untaring file...


In [38]:
len(wordidx)

400000

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [39]:
import re
from numpy.random import normal

def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and word in wordidx:
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [40]:
emb = create_emb()

In [41]:
emb.shape

(20000, 100)

In [42]:
emb_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(4, activation='softmax')])

_Note_: I started seeing lines like `4s - loss: nan - acc: 0.6783 - val_loss: nan - val_acc: 0.2131` where in the previous epoch, `val_acc` was twice that amount. A [quick search on the forums](http://forums.fast.ai/t/why-are-my-losses-nan/2931/2) surfaced this explanation:

    "There is one thing that doesn't look quite right: the final activation is not compatible with that loss function. Categorical cross-entropy expects a 'softmax' activation in the final layer, not 'sigmoid'. Consider changing that to see what happens."
    
**Categorical cross-entropy expects a `softmax` activation in the final layer, not `sigmoid`.** So I switched to `softmax`... I don't recall ever learning this information, however. Should ponder why.

In [43]:
emb_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 1000, 100)     0           embedding_input_3[0][0]          
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 1000, 100)     0           embedding_3[0][0]                
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 1000, 128)     64128       dropout_5[0][0]                  
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 1000, 128)     0           convolution1d_2[0][0]            
___________________________________________________________________________________________

In [44]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 1.4214 - acc: 0.3201 - val_loss: 1.2590 - val_acc: 0.4229
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 1.1354 - acc: 0.4382 - val_loss: 0.9398 - val_acc: 0.5117
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.9672 - acc: 0.5121 - val_loss: 0.8642 - val_acc: 0.5440
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.9053 - acc: 0.5229 - val_loss: 0.8178 - val_acc: 0.6013
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.8213 - acc: 0.5849 - val_loss: 0.7458 - val_acc: 0.6489
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.7574 - acc: 0.6326 - val_loss: 0.6981 - val_acc: 0.6868
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.7315 - acc: 0.6569 - val_loss: 0.6768 - val_acc: 0.6885
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.7101 - a

Let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [45]:
emb_model.layers[0].trainable=True

In [46]:
emb_model.optimizer.lr=1e-4

In [47]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 0.6001 - acc: 0.7380 - val_loss: 0.6175 - val_acc: 0.7401
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 0.5669 - acc: 0.7626 - val_loss: 0.5957 - val_acc: 0.7385
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.5732 - acc: 0.7487 - val_loss: 0.6030 - val_acc: 0.7401
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.5238 - acc: 0.7722 - val_loss: 0.6210 - val_acc: 0.7022
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.5008 - acc: 0.7777 - val_loss: 0.5928 - val_acc: 0.7345
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.5049 - acc: 0.7873 - val_loss: 0.5989 - val_acc: 0.7425
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.4636 - acc: 0.8060 - val_loss: 0.5808 - val_acc: 0.7506
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.4473 - a

In [48]:
emb_model.layers[0].trainable=False

In [49]:
emb_model.optimizer.lr=1e-2

In [50]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 4s - loss: 0.3964 - acc: 0.8330 - val_loss: 0.5867 - val_acc: 0.7546
Epoch 2/5
2515/2515 [==============================] - 4s - loss: 0.3622 - acc: 0.8497 - val_loss: 0.5900 - val_acc: 0.7554
Epoch 3/5
2515/2515 [==============================] - 4s - loss: 0.3425 - acc: 0.8592 - val_loss: 0.6120 - val_acc: 0.7377
Epoch 4/5
2515/2515 [==============================] - 4s - loss: 0.3464 - acc: 0.8624 - val_loss: 0.5924 - val_acc: 0.7490
Epoch 5/5
2515/2515 [==============================] - 4s - loss: 0.3467 - acc: 0.8660 - val_loss: 0.6055 - val_acc: 0.7458


Interestingly, the pretrained embeddings didn't provide any improvement...

## Pre-trained vectors + BatchNorm

In [51]:
emb = create_emb()

In [52]:
from keras.layers.normalization import BatchNormalization

batch_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    BatchNormalization(axis=1),
    Dropout(0.7),
    Dense(4, activation='softmax')])

In [53]:
batch_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
batch_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 1000, 100)     0           embedding_input_4[0][0]          
____________________________________________________________________________________________________
dropout_8 (Dropout)              (None, 1000, 100)     0           embedding_4[0][0]                
____________________________________________________________________________________________________
convolution1d_3 (Convolution1D)  (None, 1000, 128)     64128       dropout_8[0][0]                  
____________________________________________________________________________________________________
dropout_9 (Dropout)              (None, 1000, 128)     0           convolution1d_3[0][0]            
___________________________________________________________________________________________

In [54]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 2.0254 - acc: 0.2978 - val_loss: 1.4497 - val_acc: 0.3656
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 1.3942 - acc: 0.4366 - val_loss: 1.1038 - val_acc: 0.4366
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 1.1187 - acc: 0.4926 - val_loss: 1.0629 - val_acc: 0.5044
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.9667 - acc: 0.5614 - val_loss: 1.1015 - val_acc: 0.4334
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.8788 - acc: 0.6143 - val_loss: 1.0362 - val_acc: 0.5004
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.7718 - acc: 0.6573 - val_loss: 0.9625 - val_acc: 0.5997
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.7412 - acc: 0.6998 - val_loss: 0.9523 - val_acc: 0.6094
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.6608 - a

In [55]:
batch_model.layers[0].trainable=True

In [56]:
batch_model.optimizer.lr=1e-4

In [57]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 0.5179 - acc: 0.7801 - val_loss: 0.8154 - val_acc: 0.7030
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 0.5063 - acc: 0.8000 - val_loss: 0.7792 - val_acc: 0.6868
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.4787 - acc: 0.8099 - val_loss: 0.7142 - val_acc: 0.7046
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.4414 - acc: 0.8247 - val_loss: 0.6580 - val_acc: 0.7458
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.4186 - acc: 0.8266 - val_loss: 0.6574 - val_acc: 0.7458
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.3753 - acc: 0.8545 - val_loss: 0.6745 - val_acc: 0.7304
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.3674 - acc: 0.8684 - val_loss: 0.6755 - val_acc: 0.7070
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.3274 - a

In [58]:
batch_model.layers[0].trainable=False

In [59]:
batch_model.optimizer.lr=1e-2

In [60]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 4s - loss: 0.3023 - acc: 0.8823 - val_loss: 0.6212 - val_acc: 0.7337
Epoch 2/5
2515/2515 [==============================] - 4s - loss: 0.2886 - acc: 0.8915 - val_loss: 0.6952 - val_acc: 0.7006
Epoch 3/5
2515/2515 [==============================] - 4s - loss: 0.2842 - acc: 0.8938 - val_loss: 0.6426 - val_acc: 0.7320
Epoch 4/5
2515/2515 [==============================] - 4s - loss: 0.2693 - acc: 0.8918 - val_loss: 0.6414 - val_acc: 0.7312
Epoch 5/5
2515/2515 [==============================] - 4s - loss: 0.2907 - acc: 0.8843 - val_loss: 0.6604 - val_acc: 0.7304


Nada.

## Pre-trained vectors III

Let's try the model from `pretrained_word_embeddings.py (+Dropout)`

In [61]:
emb = create_emb()

In [62]:
deep_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(5),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(5),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(35), # global max pooling
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(newsgroups.target_names), activation='softmax')
])

In [63]:
deep_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
deep_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_5 (Embedding)          (None, 1000, 100)     0           embedding_input_5[0][0]          
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 1000, 100)     0           embedding_5[0][0]                
____________________________________________________________________________________________________
convolution1d_4 (Convolution1D)  (None, 996, 128)      64128       dropout_11[0][0]                 
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 996, 128)      0           convolution1d_4[0][0]            
___________________________________________________________________________________________

In [64]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 5s - loss: 1.2805 - acc: 0.3718 - val_loss: 1.1005 - val_acc: 0.4931
Epoch 2/10
2515/2515 [==============================] - 5s - loss: 0.9516 - acc: 0.4946 - val_loss: 0.9012 - val_acc: 0.5303
Epoch 3/10
2515/2515 [==============================] - 5s - loss: 0.8609 - acc: 0.5185 - val_loss: 0.8549 - val_acc: 0.5545
Epoch 4/10
2515/2515 [==============================] - 5s - loss: 0.8261 - acc: 0.5423 - val_loss: 0.8644 - val_acc: 0.5569
Epoch 5/10
2515/2515 [==============================] - 5s - loss: 0.8127 - acc: 0.5439 - val_loss: 0.8431 - val_acc: 0.5666
Epoch 6/10
2515/2515 [==============================] - 5s - loss: 0.7754 - acc: 0.5738 - val_loss: 0.7854 - val_acc: 0.5916
Epoch 7/10
2515/2515 [==============================] - 5s - loss: 0.7614 - acc: 0.5936 - val_loss: 0.7971 - val_acc: 0.5843
Epoch 8/10
2515/2515 [==============================] - 5s - loss: 0.7318 - a

In [65]:
deep_model.layers[0].trainable=True

In [66]:
deep_model.optimizer.lr=1e-4

In [67]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 5s - loss: 0.5894 - acc: 0.7368 - val_loss: 0.6439 - val_acc: 0.7345
Epoch 2/10
2515/2515 [==============================] - 5s - loss: 0.5720 - acc: 0.7392 - val_loss: 0.6288 - val_acc: 0.7433
Epoch 3/10
2515/2515 [==============================] - 5s - loss: 0.5368 - acc: 0.7594 - val_loss: 0.5995 - val_acc: 0.7522
Epoch 4/10
2515/2515 [==============================] - 5s - loss: 0.5072 - acc: 0.7841 - val_loss: 0.5994 - val_acc: 0.7522
Epoch 5/10
2515/2515 [==============================] - 5s - loss: 0.4631 - acc: 0.8087 - val_loss: 0.5876 - val_acc: 0.7393
Epoch 6/10
2515/2515 [==============================] - 5s - loss: 0.4606 - acc: 0.8008 - val_loss: 0.5710 - val_acc: 0.7627
Epoch 7/10
2515/2515 [==============================] - 5s - loss: 0.4190 - acc: 0.8215 - val_loss: 0.5977 - val_acc: 0.7538
Epoch 8/10
2515/2515 [==============================] - 5s - loss: 0.4279 - a

In [68]:
deep_model.optimizer.lr=1e-3

In [69]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 5s - loss: 0.3839 - acc: 0.8437 - val_loss: 0.6372 - val_acc: 0.7425
Epoch 2/10
2515/2515 [==============================] - 5s - loss: 0.3791 - acc: 0.8469 - val_loss: 0.5717 - val_acc: 0.7554
Epoch 3/10
2515/2515 [==============================] - 5s - loss: 0.3331 - acc: 0.8652 - val_loss: 0.5673 - val_acc: 0.7667
Epoch 4/10
2515/2515 [==============================] - 5s - loss: 0.2888 - acc: 0.8915 - val_loss: 0.5447 - val_acc: 0.7845
Epoch 5/10
2515/2515 [==============================] - 5s - loss: 0.2993 - acc: 0.8835 - val_loss: 0.5865 - val_acc: 0.7667
Epoch 6/10
2515/2515 [==============================] - 5s - loss: 0.2720 - acc: 0.9006 - val_loss: 0.5519 - val_acc: 0.7869
Epoch 7/10
2515/2515 [==============================] - 5s - loss: 0.2570 - acc: 0.9018 - val_loss: 0.5496 - val_acc: 0.7902
Epoch 8/10
2515/2515 [==============================] - 5s - loss: 0.2875 - a

Better than the first two pretrained embedding models and back to being competitive with the single conv layer with max pooling.

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [70]:
from keras.layers import Merge

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [82]:
from keras.layers import Input, Merge
from keras.models import Model

graph_in = Input((vocab_size, 100))
convs = [] 
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, border_mode='same', activation='relu')(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode='concat')(convs) 
graph = Model(graph_in, out) 

In [83]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [84]:
multi = Sequential ([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation='relu'),
    Dropout (0.7),
    Dense (len(newsgroups.target_names), activation='softmax')
    ])

In [85]:
multi.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
multi.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_7 (Embedding)          (None, 1000, 100)     2000000     embedding_input_7[0][0]          
____________________________________________________________________________________________________
dropout_19 (Dropout)             (None, 1000, 100)     0           embedding_7[0][0]                
____________________________________________________________________________________________________
model_2 (Model)                  multiple              76992       dropout_19[0][0]                 
____________________________________________________________________________________________________
dropout_20 (Dropout)             (None, 96000)         0           model_2[1][0]                    
___________________________________________________________________________________________

In [86]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 1.5174 - acc: 0.2982 - val_loss: 1.3660 - val_acc: 0.3414
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 1.2941 - acc: 0.3571 - val_loss: 1.1400 - val_acc: 0.4713
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 1.0877 - acc: 0.4664 - val_loss: 0.9286 - val_acc: 0.5593
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.9655 - acc: 0.5062 - val_loss: 0.8375 - val_acc: 0.5650
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.8699 - acc: 0.5364 - val_loss: 0.7965 - val_acc: 0.5722
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.8103 - acc: 0.5789 - val_loss: 0.7575 - val_acc: 0.6053
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.7717 - acc: 0.6076 - val_loss: 0.7344 - val_acc: 0.6602
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.6837 - a

In [87]:
multi.layers[0].trainable=False

In [88]:
multi.optimizer.lr=1e-5

In [89]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 0.4920 - acc: 0.7917 - val_loss: 0.5570 - val_acc: 0.7627
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 0.4411 - acc: 0.8254 - val_loss: 0.5288 - val_acc: 0.7789
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.4075 - acc: 0.8294 - val_loss: 0.5321 - val_acc: 0.7845
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.3489 - acc: 0.8581 - val_loss: 0.5092 - val_acc: 0.7918
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.3256 - acc: 0.8799 - val_loss: 0.5326 - val_acc: 0.7950
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.2921 - acc: 0.8867 - val_loss: 0.5151 - val_acc: 0.7958
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.2561 - acc: 0.9046 - val_loss: 0.5485 - val_acc: 0.7998
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.2361 - a

In [90]:
multi.layers[0].trainable=True

In [91]:
multi.optimizer.lr=1e-2

In [92]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 7s - loss: 0.1830 - acc: 0.9312 - val_loss: 0.5532 - val_acc: 0.8152
Epoch 2/5
2515/2515 [==============================] - 7s - loss: 0.1925 - acc: 0.9245 - val_loss: 0.5283 - val_acc: 0.8241
Epoch 3/5
2515/2515 [==============================] - 7s - loss: 0.1598 - acc: 0.9392 - val_loss: 0.5557 - val_acc: 0.8111
Epoch 4/5
2515/2515 [==============================] - 7s - loss: 0.1668 - acc: 0.9360 - val_loss: 0.5427 - val_acc: 0.8184
Epoch 5/5
2515/2515 [==============================] - 7s - loss: 0.1511 - acc: 0.9479 - val_loss: 0.5419 - val_acc: 0.8329


Highest deep-learning result so far! And the most comparable to the 'shallow' bag-of-words results that achieved upper-80s.

Bonus: this was also the least stressful to watch train because `val_acc` (generally) continued to rise instead of bouncing around like the other models.